<center>
<img src="../../img/ods_stickers.jpg">
## Open Machine Learning Course
<center>
Author: Yury Kashnitsky, Data Scientist at Mail.Ru Group

This material is subject to the terms and conditions of the license [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Free use is permitted for any non-comercial purpose with an obligatory indication of the names of the authors and of the source.

## <center>Assignment #6. Part 1
### <center> Beating benchmarks in "Catch Me If You Can: Intruder Detection through Webpage Session Tracking"
    
[Competition](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2). The task is to beat "Assignment 6 baseline".

In [ ]:
import seaborn as sns
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from matplotlib import pyplot as plt

Reading original data

In [ ]:
PATH_TO_DATA = ('../../data')
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'), index_col='session_id')
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'), index_col='session_id')

In [ ]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)


In [ ]:
train_df.head()

Separate target feature 

In [ ]:
y_train = train_df['target']
train_df.drop('target',axis=1)

Build Tf-Idf features based on sites. You can use `ngram_range`=(1, 3) and `max_features`=100000 or more

In [ ]:
import pickle

sites = ['site%s' % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

# Load websites dictionary
with open(os.path.join(PATH_TO_DATA, 'site_dic.pkl'), 'rb') as input_file:
    site_dict = pickle.load(input_file)

# Create dataframe for the dictionary
sites_dict = pd.DataFrame(list(site_dict.keys()), index=list(site_dict.values()), columns=['site'])
sites_dict.head()

In [ ]:
def vec(train, test):
    def trans(x):
        return ' '.join(x)
    
    train_sites = train.apply(lambda x: x.map(sites_dict['site'])).fillna('')
    train_sites = train_sites.apply(lambda x: trans(x), axis=1)
    test_sites = test.apply(lambda x: x.map(sites_dict['site'])).fillna('')
    test_sites = test_sites.apply(lambda x: trans(x), axis=1)
    
    vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=100000)
    vectorizer.fit(pd.concat([train_sites, test_sites]))
    
    return vectorizer.transform(train_sites), vectorizer.transform(test_sites)

In [ ]:
X_train_sites, X_test_sites = vec(train_df[sites], test_df[sites])

Add features based on the session start time: hour, whether it's morning, day or night and so on.

Scale this features and combine then with Tf-Idf based on sites (you'll need `scipy.sparse.hstack`)

In [78]:
times = ['time%s' % i for i in range(1, 11)]

#train_df[times] = train_df[times].fillna(pd.to_datetime('2000-01-01 00:00:00'))
#test_df[times] = test_df[times].fillna(pd.to_datetime('2000-01-01 00:00:00'))

def full_times(train_times, test_times):
    
    train_times_ = pd.DataFrame(np.zeros((train_times.shape[0],3), dtype=bool))
    test_times_ = pd.DataFrame(train_times_)

    for n in times:
        #train
        data = np.array(pd.DatetimeIndex(train_times[n]).hour)
        train_times_[0] = np.where((train_times_[0])|(data>=0)&(data<8),1,0)
        train_times_[1] = np.where((train_times_[1])|(data>=8)&(data<16),1,0)
        train_times_[2] = np.where((train_times_[2])|(data>=16)&(data<24),1,0)
        #test
        data = np.array(pd.DatetimeIndex(test_times[n]).hour)
        test_times_[0] = np.where((test_times_[0])|(data>=0)&(data<8),1,0)
        test_times_[1] = np.where((test_times_[1])|(data>=8)&(data<16),1,0)
        test_times_[2] = np.where((test_times_[2])|(data>=16)&(data<24),1,0)
        
    return train_times_, test_times_ 

In [ ]:
train_times = train_df[times][0:10]

train_times_ = pd.DataFrame(np.zeros((train_times.shape[0],3), dtype=bool))

for n in times:
    #train
    data = np.array(pd.DatetimeIndex(train_times[n]).hour)
    train_times_[0] = np.where((train_times_[0])|(data>=0)&(data<8),1,0)
    train_times_[1] = np.where((train_times_[1])|(data>=8)&(data<16),1,0)
    train_times_[2] = np.where((train_times_[2])|(data>=16)&(data<24),1,0)

train_times_

#X_train_times, X_test_times = full_times(train_df[times][0:3], test_df[times][0:3]) 

In [79]:
X_train = csr_matrix(hstack([X_train_sites, X_train_times]))
X_test = csr_matrix(hstack([X_test_sites, X_test_times]))

Perform cross-validation with logistic regression.

In [80]:
from sklearn.model_selection import cross_val_score

lr = LogisticRegression(C=0.5, random_state=17, n_jobs=-1, class_weight='balanced')

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

cv_aucs = cross_val_score(lr, X_train, y_train, scoring="roc_auc", cv=skf)

print(np.mean(cv_aucs))

0.985201955153


Make prediction for the test set and form a submission file.

In [81]:
lr.fit(X_train, y_train)
test_pred = lr.predict_proba(X_test)[:, 1]

In [82]:
write_to_submission_file(test_pred, os.path.join(PATH_TO_DATA, "assignment6_alice_submission.csv"))